# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [2]:
import pandas as pd
import requests
import json
import os
from azureml.core import Workspace, Datastore, Dataset, Experiment, Environment
from azureml.train.automl import AutoMLConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.model import Model, InferenceConfig
from azureml.widgets import RunDetails
from azureml.core.webservice import LocalWebservice
from azureml.core.webservice import AciWebservice

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
# Read and clean the data

raw_data = pd.read_csv('data-unbalanced.csv')
raw_data = raw_data.dropna()
raw_data = raw_data.drop("WO-MRR NUM", axis=1)
raw_data = raw_data.drop("Has MRR", axis=1)
raw_data = pd.get_dummies(raw_data, columns=['PROD_CDE','PREP ASSOCIATE', 'FURN'])
print(raw_data.shape)

(26038, 91)


In [4]:
# Create a new Dataset from the raw_data
ws = Workspace.from_config()
datastore = Datastore.get(ws, 'workspaceblobstore')

dataset = Dataset.Tabular.register_pandas_dataframe(raw_data, datastore ,"Sinter", show_progress=True)

Method register_pandas_dataframe: This is an experimental method, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/009d7d87-4406-4a80-9f39-38827bae11aa/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


In [5]:
# choose a name for experiment
experiment_name = 'Sinter_AML'

experiment=Experiment(ws, experiment_name)

# Create a Compute target

In [6]:
cpu_cluster_name = "ML-Cluster"
compute_config = AmlCompute.provisioning_configuration("Standard_D2_V2", min_nodes=0, max_nodes=2)
cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
cpu_cluster.wait_for_completion(show_output=True)

InProgress....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [7]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}
automl_config = AutoMLConfig(compute_target=cpu_cluster,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="HAS SCRAP AT SINTER",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [8]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)
RunDetails(remote_run).show()

Submitting remote run.
No run_configuration provided, running on ML-Cluster with default configuration
Running on remote compute: ML-Cluster


Experiment,Id,Type,Status,Details Page,Docs Page
Sinter_AML,AutoML_f1aa0e79-74ff-4535-ad1e-b89e2396e46a,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Train-Test data split
STATUS:       DONE
DESCRIPTION:  Your input data has been split into a training dataset and a holdout test dataset for validation of the model. The test holdout dataset reflects the original distribution of your input data.
              
DETAILS:      
+---------------------------------+---------------------------------+---------------------------------+
|Dataset                          |Row counts                       |Percentage                       |
+=================================+=================================+===============

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [9]:
best_run, best_model = remote_run.get_output()

best_model_metrics = best_run.get_metrics()


In [10]:
# Display the metrics of the best model

metrics = pd.DataFrame.from_dict(best_model_metrics, orient='index', columns=['metric value'])
metrics

,metric value
recall_score_macro,0.65
average_precision_score_macro,0.59
accuracy,0.75
average_precision_score_weighted,0.89
recall_score_weighted,0.75
matthews_correlation,0.20
AUC_macro,0.72
recall_score_micro,0.75
norm_macro_recall,0.30
precision_score_weighted,0.87


In [11]:
#TODO: Save the best model

aml_model = remote_run.register_model(model_name='AML_Best', tags=best_model_metrics)



## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [37]:
# Define inference config

env = best_run.get_environment()
inference_config = InferenceConfig(
    environment= env,
    source_directory = "./source_dir",
    entry_script="score.py"
)

In [38]:
# Define a deployment configuration on the Webservice

deploy_config = AciWebservice.deploy_configuration(
    cpu_cores=0.5, memory_gb=1, auth_enabled=True, enable_app_insights=True
)

In [44]:
# Deploy the model

service = Model.deploy(
    workspace = ws,
    name = "sinter-scrap-model1",
    models = [aml_model],
    inference_config = inference_config,
    deployment_config = deploy_config,
    overwrite = True
)

service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-10-19 13:09:01+00:00 Creating Container Registry if not exists.
2021-10-19 13:09:01+00:00 Registering the environment.
2021-10-19 13:09:01+00:00 Use the existing image.
2021-10-19 13:09:02+00:00 Generating deployment configuration.
2021-10-19 13:09:03+00:00 Submitting deployment to compute.
2021-10-19 13:09:05+00:00 Checking the status of deployment sinter-scrap-model1.

In [15]:
print(service.get_logs())

2021-10-19T11:38:31,334573300+00:00 - iot-server/run 
2021-10-19T11:38:31,347629600+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2021-10-19T11:38:31,514118200+00:00 - rsyslog/run 
2021-10-19T11:38:32,097911700+00:00 - nginx/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-10-19T11:38:32,817333100+00:00 - iot-server/finish 1 0
2021-10-19T11:38:32,822030900+00:00 - Exit code 1 is normal. Not restarting iot-server.
rsyslogd: /azureml-envs/azureml_1b22a9c5d9b447e8fbdcc34171e02fe8/lib/libuuid.so.1: no version information available (required by rsyslogd)
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (64)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 90
SPARK_HOME not set. Skipping PySpark Initialization.
Generating new fontManager, this may take some time...
Initializing logger
2021-10-19 11:38:37,154 | root | INFO | Starting up app insights client
logging socket was 

In [16]:
print(service.scoring_uri)

http://4a97e38d-b187-4a3c-8b8a-33986706b2ca.southcentralus.azurecontainer.io/score


TODO: In the cell below, send a request to the web service you deployed to test it.

In [42]:
import json 
import numpy as np

test_data = raw_data[0:1]
data = ({'data':test_data.to_dict(orient='records')})
sample = json.dumps(data)

#json_data = json.loads(sample)['data']

#pd_data = pd.DataFrame(json_data)

#print(pd_data.shape)

#np_array = np.array(json_data)

#len(np_array)

print(service.run(sample))

ERROR:azureml.core.webservice.aci:Received bad response from service. More information can be found by calling `.get_logs()` on the webservice object.
Response Code: 502
Headers: {'Connection': 'keep-alive', 'Content-Length': '42', 'Content-Type': 'text/html; charset=utf-8', 'Date': 'Tue, 19 Oct 2021 13:04:56 GMT', 'Server': 'nginx/1.14.0 (Ubuntu)', 'X-Ms-Request-Id': '20d62c9f-471b-4c54-ad13-5c703d0f07c1', 'X-Ms-Run-Function-Failed': 'True'}
Content: b'DataFrame constructor not properly called!'



WebserviceException: WebserviceException:
	Message: Received bad response from service. More information can be found by calling `.get_logs()` on the webservice object.
Response Code: 502
Headers: {'Connection': 'keep-alive', 'Content-Length': '42', 'Content-Type': 'text/html; charset=utf-8', 'Date': 'Tue, 19 Oct 2021 13:04:56 GMT', 'Server': 'nginx/1.14.0 (Ubuntu)', 'X-Ms-Request-Id': '20d62c9f-471b-4c54-ad13-5c703d0f07c1', 'X-Ms-Run-Function-Failed': 'True'}
Content: b'DataFrame constructor not properly called!'
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Received bad response from service. More information can be found by calling `.get_logs()` on the webservice object.\nResponse Code: 502\nHeaders: {'Connection': 'keep-alive', 'Content-Length': '42', 'Content-Type': 'text/html; charset=utf-8', 'Date': 'Tue, 19 Oct 2021 13:04:56 GMT', 'Server': 'nginx/1.14.0 (Ubuntu)', 'X-Ms-Request-Id': '20d62c9f-471b-4c54-ad13-5c703d0f07c1', 'X-Ms-Run-Function-Failed':

In [43]:
service.get_logs()

'2021-10-19T13:03:18,233826700+00:00 - nginx/run \n2021-10-19T13:03:18,245304000+00:00 - iot-server/run \n2021-10-19T13:03:18,314432800+00:00 - rsyslog/run \n2021-10-19T13:03:18,313497200+00:00 - gunicorn/run \nDynamic Python package installation is disabled.\nStarting HTTP server\nrsyslogd: /azureml-envs/azureml_1b22a9c5d9b447e8fbdcc34171e02fe8/lib/libuuid.so.1: no version information available (required by rsyslogd)\nEdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...\n2021-10-19T13:03:19,027141200+00:00 - iot-server/finish 1 0\n2021-10-19T13:03:19,037201000+00:00 - Exit code 1 is normal. Not restarting iot-server.\nStarting gunicorn 20.1.0\nListening at: http://127.0.0.1:31311 (64)\nUsing worker: sync\nworker timeout is set to 300\nBooting worker with pid: 89\nSPARK_HOME not set. Skipping PySpark Initialization.\nGenerating new fontManager, this may take some time...\nInitializing logger\n2021-10-19 13:03:22,706 | root | INFO | Starting up app insights client\

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
